<a href="https://colab.research.google.com/github/Aadik1ng/Skin_cell_classification-using-CNN/blob/main/Skin_Cell_detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import tensorflow as tf
from tensorflow import keras
import gc
import random
import os
import pandas as pd
import json
import zipfile
import os
import numpy as np
import tensorflow as tf
from PIL import Image
import numpy as np
import pandas as pd

In [2]:
!mkdir ~/.kaggle

!touch ~/.kaggle/kaggle.json

api_token = {"username":"adik1ng","key":"b172f9e6f987c3e819a9e2876f26881a"}



with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

skin-cancer-mnist-ham10000.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
%%time

# Unzip the dataset
zip = '/content/skin-cancer-mnist-ham10000.zip'
zip_ref = zipfile.ZipFile(zip, 'r')
zip_ref.extractall('images')
zip_ref.close()

In [2]:
cancer=pd.read_csv("/content/images/HAM10000_metadata.csv")

In [3]:
cancer['age'].fillna('mean',inplace=True)

In [4]:
paths=[]
for arw in os.listdir('/content/images'):
  if (arw[:10]=='ham10000_i'):
      inside=os.path.join('/content/images',arw)
      for imagearw in os.listdir(inside):
        paths.append(os.path.join(inside,imagearw))

In [5]:
ids_paths = {os.path.splitext(os.path.basename(path))[0]:path for path in paths}
cancer['path']=cancer['image_id'].map(ids_paths)


In [6]:
labels=cancer['dx'].to_frame()
labels.describe()
label=pd.get_dummies(labels)

In [7]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the column
encoded_labels = label_encoder.fit_transform(labels)




/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
cancer['encoded_labels']=encoded_labels

In [9]:
cancer.drop(['lesion_id','image_id','path','dx'],inplace=True,axis=1)

In [10]:
cancer.head()

,dx_type,age,sex,localization,encoded_labels
0,histo,80.0,male,scalp,2
1,histo,80.0,male,scalp,2
2,histo,80.0,male,scalp,2
3,histo,80.0,male,scalp,2
4,histo,75.0,male,ear,2


In [11]:
X = cancer.drop('encoded_labels', axis=1)  # Assuming 'target' is the column name of the target variable
y = cancer['encoded_labels']

In [12]:
import pandas as pd
df = pd.read_csv('/content/images/hmnist_28_28_RGB.csv')
y = df['label']
x = df.drop(columns = ['label'])
x.shape

(10015, 2352)

In [29]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [19]:
# Load pixel values from pixel_df
pixel_data =df.drop('label', axis=1)  # Assuming 'target' is the column name of the target variable
pixel_data = pixel_data.values
pixel_data = pixel_data.reshape(-1, 28, 28, 3)
pixel_data=pixel_data/255
# Convert the target labels to one-hot encoding
target_labels =label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pixel_data, target_labels, test_size=0.2)


In [ ]:
target_labels

In [20]:
num_classes=7
model = tf.keras.Sequential()

model.add(layers.Conv2D(256, kernel_size=(3, 3), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
model.add(tf.keras.layers.Conv2D(20,(3,3),strides=(1,1),padding='valid',activation='relu'))
model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
model.add(tf.keras.layers.GroupNormalization(groups=3))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
model.add(tf.keras.layers.Conv2D(10,(3,3),strides=(1,1),padding='valid',activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Normalization())

model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(7,activation='softmax'))
model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 256)       7168      
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 30)        69150     
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 30)       120       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 30)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 20)        5420      
                                                                 
 conv2d_3 (Conv2D)           (None, 9, 9, 15)          2

In [22]:
from tensorflow.keras.callbacks import LearningRateScheduler

# Define a function that adjusts the learning rate
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0001
    else:
        return 0.00001

# Create the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
%%time

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
with tf.device('/GPU:0'):
  model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),callbacks=[lr_scheduler])


Epoch 1/50
251/251 [==============================] - 12s 14ms/step - loss: 0.9982 - accuracy: 0.6584 - val_loss: 1.2328 - val_accuracy: 0.6495 - lr: 0.0010
Epoch 2/50
251/251 [==============================] - 3s 10ms/step - loss: 0.8834 - accuracy: 0.6887 - val_loss: 1.1411 - val_accuracy: 0.6221 - lr: 0.0010
Epoch 3/50
251/251 [==============================] - 3s 13ms/step - loss: 0.8415 - accuracy: 0.6950 - val_loss: 1.1806 - val_accuracy: 0.6011 - lr: 0.0010
Epoch 4/50
251/251 [==============================] - 5s 18ms/step - loss: 0.8002 - accuracy: 0.7077 - val_loss: 0.8755 - val_accuracy: 0.6865 - lr: 0.0010
Epoch 5/50
251/251 [==============================] - 5s 18ms/step - loss: 0.7781 - accuracy: 0.7169 - val_loss: 0.8612 - val_accuracy: 0.6895 - lr: 0.0010
Epoch 6/50
251/251 [==============================] - 5s 20ms/step - loss: 0.7406 - accuracy: 0.7288 - val_loss: 0.9068 - val_accuracy: 0.6780 - lr: 0.0010
Epoch 7/50
251/251 [==============================] - 3s 10ms/s

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [39]:
Y_pred_labels=model.predict(X_test)

63/63 [==============================] - 1s 4ms/step
